In [4]:
import json
import base64
import requests
import tensorflow as tf

# URL Lokal Docker
URL = "http://localhost:8501/v1/models/churn-model:predict"

# --- 1. Helper Function untuk membuat tf.train.Example ---
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value.encode()]))

def _float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

# --- 2. Siapkan Data Dummy (Fixed Version) ---
example = tf.train.Example(features=tf.train.Features(feature={
    # [FIX] Tambahkan customerID karena Schema mewajibkannya (Isi bebas)
    'customerID': _bytes_feature("testing_id_001"),

    'gender': _bytes_feature("Female"),
    'SeniorCitizen': _int64_feature(0),
    'Partner': _bytes_feature("Yes"),
    'Dependents': _bytes_feature("No"),
    'tenure': _int64_feature(1),
    'PhoneService': _bytes_feature("No"),
    'MultipleLines': _bytes_feature("No phone service"),
    'InternetService': _bytes_feature("DSL"),
    'OnlineSecurity': _bytes_feature("No"),
    'OnlineBackup': _bytes_feature("Yes"),
    'DeviceProtection': _bytes_feature("No"),
    'TechSupport': _bytes_feature("No"),
    'StreamingTV': _bytes_feature("No"),
    'StreamingMovies': _bytes_feature("No"),
    'Contract': _bytes_feature("Month-to-month"),
    'PaperlessBilling': _bytes_feature("Yes"),
    'PaymentMethod': _bytes_feature("Electronic check"),
    
    # Float tetap Float
    'MonthlyCharges': _float_feature(29.85), 
    
    # [FIX] TotalCharges wajib String
    'TotalCharges': _bytes_feature("29.85")  
}))

# --- 3. Serialize & Encode ke Base64 ---
serialized_example = example.SerializeToString()
b64_example = base64.b64encode(serialized_example).decode('utf-8')

# --- 4. Susun Payload JSON ---
payload = {
    "signature_name": "serving_default",
    "instances": [
        {
            "examples": {"b64": b64_example}
        }
    ]
}

# --- 5. Kirim Request ---
try:
    response = requests.post(URL, json=payload)
    result = response.json()
    
    if response.status_code == 200:
        print("✅ Request Berhasil!")
        print("Hasil Prediksi:", result)
        
        prediction = result['predictions'][0][0]
        print(f"Nilai Probability: {prediction:.4f}")
        print(f"Keputusan: {'CHURN' if prediction > 0.5 else 'TIDAK CHURN'}")
    else:
        print("❌ Request Gagal:", response.text)

except Exception as e:
    print("Error Connection:", e)

Request Gagal: {
    "error": "Name: <unknown>, Feature: customerID (data type: string) is required but could not be found.\n\t [[{{function_node __inference_serve_tf_examples_fn_43684}}{{node ParseExample/ParseExampleV2}}]]"
}
